# Ánalise Comparativa

* **Importando Bibliotecas**

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import joblib

#### **Leitura dos Dados**

In [26]:
df= pd.read_csv("../data/raw/wines_SPA.csv")
df.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [27]:
df_dict = pd.read_csv('../data/external/dictionary.csv')
df_dict

,variavel,descricao,tipo,subtipo
0,winery,Nome da vinícola,qualitativa,nominal
1,wine,Nome do vinho,qualitativa,nominal
2,year,Ano em que as uvas foram colhidas,quantitativa,discreta
3,rating,Avaliação média dada ao vinho pelos usuários [...,quantitativa,contínua
4,num_reviews,Número de usuários que avaliaram o vinho,quantitativa,discreta
5,country,País de origem [Espanha],inútil,inútil
6,region,Região do vinho,qualitativa,nominal
7,price,Preço em euros [€],quantitativa,contínua
8,type,Variedade de vinho,qualitativa,nominal
9,body,"Pontuação de corpo, definida como a riqueza e ...",qualitativa,ordinal


#### **Renomeando as colunas**

In [28]:
mapeamento_colunas = {
    'winery': 'vinícola',
    'wine': 'vinho',
    'year': 'ano',
    'rating': 'avaliação',
    'num_reviews': 'num_avaliações',
    'country': 'país',
    'region': 'região',
    'price': 'preço',
    'type': 'tipo',
    'body': 'corpo',
    'acidity': 'acidez'
}

In [29]:
def renomear_colunas(df, mapping):
    df.rename(columns=mapping, inplace=True)
    return df

In [30]:
df_dict['variavel'] = df_dict['variavel'].replace(mapeamento_colunas)
df = renomear_colunas(df, mapeamento_colunas)
df.columns

Index(['vinícola', 'vinho', 'ano', 'avaliação', 'num_avaliações', 'país',
       'região', 'preço', 'tipo', 'corpo', 'acidez'],
      dtype='object')

## Tratamento e Transformação de dados

#### **Verificando os Valores Faltantes e Dados Duplicados**

In [31]:
print(df.duplicated().sum())

5452


In [32]:
df = df.drop(df[df.duplicated()].index, axis=0)
df.reset_index()
print(f'Number of Duplicates: {df.duplicated().sum()}')

Number of Duplicates: 0


In [33]:
df['ano'] = df['ano'].replace('N.V.', np.nan)

In [34]:
print(f'Verificando valores faltantes : \n\n{df.isna().sum()}')

Verificando valores faltantes : 

vinícola            0
vinho               0
ano                72
avaliação           0
num_avaliações      0
país                0
região              0
preço               0
tipo              106
corpo             271
acidez            271
dtype: int64


* **Verificando os Valores discrepantes**

In [35]:
# Função para identificar outliers usando o IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # Primeiro quartil
    Q3 = df[column].quantile(0.75)  # Terceiro quartil
    IQR = Q3 - Q1                     # Intervalo Interquartil
    
    lower_bound = Q1 - 1.5 * IQR      # Limite inferior
    upper_bound = Q3 + 1.5 * IQR      # Limite superior
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [36]:
# Detectando outliers apenas para colunas numéricas
outliers_summary = []

for col in df.select_dtypes(include=[np.number]).columns:  # Filtra apenas colunas numéricas
    outliers = detect_outliers_iqr(df, col)
    if not outliers.empty:
        outliers_summary.append({'Coluna': col, 'Quantidade de Outliers': len(outliers)})

outliers_df = pd.DataFrame(outliers_summary)

print(outliers_df)

           Coluna  Quantidade de Outliers
0       avaliação                       2
1  num_avaliações                     242
2           preço                     245
3           corpo                      34
4          acidez                     105


# Pré-Processamento dos Dados

In [37]:
# Variável alvo
target_variable = 'preço'

# Variáveis inúteis
useless_variables = (
    df_dict
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)

# Variáveis nominais
nominal_variables = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis ordinais
ordinal_variables = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis contínuas
continuous_variables = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis discretas
discrete_variables = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [38]:
X = df.drop(columns=[target_variable] + useless_variables)
y = df[target_variable]

In [39]:
# Exibindo as categorias de variáveis
print("Nominal Variables:", nominal_variables)
print("Ordinal Variables:", ordinal_variables)
print("Continuous Variables:", continuous_variables)
print("Discrete Variables:", discrete_variables)

Nominal Variables: ['vinícola', 'vinho', 'região', 'tipo']
Ordinal Variables: ['corpo', 'acidez']
Continuous Variables: ['avaliação']
Discrete Variables: ['ano', 'num_avaliações']


O código acima organiza e prepara os dados de forma sistemática para análise ou modelagem. Ele facilita a aplicação de diferentes técnicas a cada tipo de variável, como codificação (para nominais) ou normalização (para contínuas)

#### **Pipelines de pré-processamento por tipo de variável**

In [40]:
# Nominal variables: imputação de valores faltantes e codificação one-hot
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="most_frequent")),  # Preenchimento com o valor mais frequente
    ("encoding", OneHotEncoder(sparse_output=False))  # Codificação One-Hot
])

# Ordinal variables: imputação de valores faltantes e codificação ordinal
ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="median")),  # Preenchimento com a mediana
    ("encoding", OrdinalEncoder())  # Codificação ordinal
])

# Continuous variables: imputação de valores faltantes e normalização
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="mean")),  # Preenchimento com a média
    ("normalization", StandardScaler())  # Normalização
])

# Discrete variables: imputação de valores faltantes e normalização
discrete_preprocessor = Pipeline(steps=[
    ("missing", KNNImputer()),  # Preenchimento baseado em K vizinhos mais próximos
    ("normalization", StandardScaler())  # Normalização
])

Esse código acima organiza pipelines para pré-processar variáveis diferentes (nominais, ordinais, contínuas e discretas). Cada tipo de variável tem necessidades específicas, como tratamento de valores faltantes ou codificação.

In [41]:
# Criação do ColumnTransformer para processar cada tipo de variável
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),   
    ("ordinal", ordinal_preprocessor, ordinal_variables),   
    ("continuous", continuous_preprocessor, continuous_variables),  
    ("discrete", discrete_preprocessor, discrete_variables), 
])

As variáveis nominais, contínuas, ordinais e discretas são extraídas do dicionário de dados com base no subtipo. Isso assegura que cada tipo de dado seja tratado com o pipeline apropriado.

In [42]:
preprocessor

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['vinícola', 'vinho', 'região', 'tipo']),
                                ('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='median')),
                                                 ('encoding',
                                                  OrdinalEncoder())]),
                                 ['corpo', 'acidez']),
                                ('continuous',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['avaliação']),
                                ('discrete',
                                 Pipeline(steps=[('missing', KNNImputer()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['ano', 'num_avaliações'])])

#### **Aplicando o preprocessor aos dados**

In [43]:
X_preprocessed = preprocessor.fit_transform(X)

In [44]:
np.set_printoptions(precision=3, suppress=True)

print(X_preprocessed)

[[ 0.     0.     0.    ...  3.394  0.167 -0.375]
 [ 0.     0.     0.    ...  3.394  0.62  -0.395]
 [ 0.     0.     0.    ...  2.714 -0.196  0.886]
 ...
 [ 0.     0.     0.    ... -1.368  0.439 -0.135]
 [ 0.     0.     0.    ... -1.368 -0.559 -0.138]
 [ 0.     0.     0.    ... -1.368  0.53  -0.111]]
